In [1]:
!pip install pandas openpyxl xlrd

Defaulting to user installation because normal site-packages is not writeable


In [7]:
#Pendants

In [12]:
import pandas as pd

# --- Step 1: Read Excel and skip top 5 rows ---
file_path = r"C:\Users\Admin\Downloads\Mor_style_men.xlsx"
df = pd.read_excel(file_path, skiprows=4)

selected_columns = ['SAP CODE', 'Shimayra #', 'LABEL DESCRIPTION', 'Diamond Quality','ORDER QTY']
df = df[selected_columns].copy()

# --- Step 3: Rename columns ---
df.rename(columns={
    'SAP CODE': 'SKUNo',
    'Shimayra #': 'StyleCode',
    'LABEL DESCRIPTION': 'CustomerProductionInstruction',
    'ORDER QTY': 'OrderQty'
}, inplace=True)

df

,SKUNo,StyleCode,CustomerProductionInstruction,Diamond Quality,OrderQty
0,S.89194094,PD0000657D,Christ Necklace OR585 White Gold Diamond 0.25 ...,G/H SI3 0.25CT,5


In [9]:
# import pandas as pd

# # --- Step 1: Read Excel and skip top 5 rows ---
# file_path = r"C:\Users\Admin\Downloads\Mor_style_men2.xlsx"
# df = pd.read_excel(file_path, skiprows=4)

# selected_columns = ['SAP CODE', 'Shimayra #', 'LABEL DESCRIPTION', 'Diamond Quality']
# df = df[selected_columns].copy()

# df

,SAP CODE,Shimayra #,LABEL DESCRIPTION,Diamond Quality
0,S.B2B08358,RG0002939A-EU58WG,Christ Solitaire Rings OR585 White Gold Diamon...,DIAMOND K/L SI3 0.1CT


In [14]:
#PEN MOR FINAL CODE
import pandas as pd
import re

# --- Step 1: Read Excel and skip top 5 rows ---
#file_path = r"C:\Users\Admin\Downloads\Mor_style_men.xlsx"
file_path = r"C:\Users\Admin\Downloads\Mor_style_men2.xlsx"
df = pd.read_excel(file_path, skiprows=4)

selected_columns = ['SAP CODE', 'Shimayra #', 'LABEL DESCRIPTION', 'Diamond Quality', 'ORDER QTY']
df = df[selected_columns].copy()

# --- Step 3: Rename columns ---
df.rename(columns={
    'SAP CODE': 'SKUNo',
    'Shimayra #': 'StyleCode',
    'LABEL DESCRIPTION': 'CustomerProductionInstruction',
    'ORDER QTY': 'OrderQty'
}, inplace=True)


# --- Helper Functions ---
def map_metal_from_instruction(instruction):
    """Map metal type from CustomerProductionInstruction"""
    if pd.isna(instruction):
        return ""
    
    instruction = str(instruction).upper()
    
    # Check for OR585 variations
    if "OR585 WHITE GOLD" in instruction:
        return "G585W"
    elif "OR585 YELLOW GOLD" in instruction:
        return "G585Y"
    elif "OR585 PINK GOLD" in instruction:
        return "G585P"
    # Check for OR587 variations
    elif "OR587 WHITE GOLD" in instruction:
        return "G587W"
    elif "OR587 YELLOW GOLD" in instruction:
        return "G587Y"
    elif "OR587 PINK GOLD" in instruction:
        return "G587P"
    # Check for OR750 variations
    elif "OR750 WHITE GOLD" in instruction:
        return "G750W"
    elif "OR750 YELLOW GOLD" in instruction:
        return "G750Y"
    elif "OR750 PINK GOLD" in instruction:
        return "G750P"
    # Check for Platinum
    elif "PT" in instruction or "PLATINUM" in instruction:
        return "PC95"
    else:
        return ""


def extract_tone_from_metal(metal):
    """Extract tone from metal code"""
    if pd.isna(metal) or metal == "":
        return ""
    
    metal = str(metal)
    if metal.endswith("W"):
        return "W"
    elif metal.endswith("Y"):
        return "Y"
    elif metal.endswith("P"):
        return "P"
    elif metal == "PC95":
        return "PT"
    else:
        return ""


def create_special_remarks(row):
    """Create SpecialRemarks from SKUNo, Metal, Tone, and Diamond Quality"""
    sku = str(row.get('SKUNo', "")) if pd.notna(row.get('SKUNo', "")) else ""
    metal = str(row.get('Metal', "")) if pd.notna(row.get('Metal', "")) else ""
    tone = str(row.get('Tone', "")) if pd.notna(row.get('Tone', "")) else ""
    diamond_quality = str(row.get('Diamond Quality', "")) if pd.notna(row.get('Diamond Quality', "")) else ""
    
    # Extract metal number (e.g., "585" from "G585W")
    metal_num = ""
    if metal:
        numbers = re.findall(r'\d+', metal)
        if numbers:
            metal_num = numbers[0]
    
    # Map tone to full name
    tone_map = {
        "W": "WHITE GOLD",
        "Y": "YELLOW GOLD", 
        "P": "PINK GOLD",
        "PT": "PLATINUM"
    }
    tone_full = tone_map.get(tone, tone)
    
    # Create the special remarks
    parts = []
    if sku:
        parts.append(f"S.{sku}")
    if metal_num:
        parts.append(metal_num)
    if tone_full:
        parts.append(tone_full)
    if diamond_quality:
        parts.append(f"DIA QLTY: {diamond_quality}")
    
    return ", ".join(parts)


def get_design_production_instruction(tone):
    """Get design production instruction based on tone"""
    if pd.isna(tone) or tone == "":
        return ""
    tone = str(tone)
    if tone == "W":
        return "WHITE RODIUM"
    else:
        return "NO RODIUM"


def get_stamp_instruction(metal):
    """Get stamp instruction based on metal"""
    if pd.isna(metal) or metal == "":
        return ""
    
    metal = str(metal)
    numbers = re.findall(r'\d+', metal)
    if numbers:
        metal_num = numbers[0]
        return f"GOLD TITLE ({metal_num} with frame) + SJ LOGO + CHR with \"ROUND FRAME\" LOGO"
    else:
        return "GOLD TITLE + SJ LOGO + CHR with \"ROUND FRAME\" LOGO"


# --- Get user inputs ---
print("Please provide the following inputs:")
item_po_no = input("ItemPoNo: ").strip()
priority = input("Priority: ").strip()


# --- Create the final DataFrame ---
result_df = pd.DataFrame()

# 1. SrNo
result_df['SrNo'] = range(1, len(df) + 1)

# 2. StyleCode
result_df['StyleCode'] = df['StyleCode']

# 3. ItemSize (blank)
result_df['ItemSize'] = ""

# 4. OrderQty
result_df['OrderQty'] = df['OrderQty']

# 5. OrderItemPcs (blank)
result_df['OrderItemPcs'] = ""

# 6. Metal (mapped)
result_df['Metal'] = df['CustomerProductionInstruction'].apply(map_metal_from_instruction)

# 7. Tone (from Metal)
result_df['Tone'] = result_df['Metal'].apply(extract_tone_from_metal)

# 8. ItemPoNo (input)
result_df['ItemPoNo'] = item_po_no

# 9. ItemRefNo (blank)
result_df['ItemRefNo'] = ""

# 10. StockType (blank)
result_df['StockType'] = ""

# 11. Priority (input)
result_df['Priority'] = priority

# 12. MakeType (blank)
result_df['MakeType'] = ""

# 13. CustomerProductionInstruction
def format_instruction_with_eu(text):                                #EU part 
    if pd.isna(text):
        return ""
    text = str(text).strip()
    # Match if ends with two digits (like "58" or "52")
    match = re.search(r'\b(\d{2})$', text)
    if match:
        size = match.group(1)
        # Replace the ending number with EU + number
        text = re.sub(r'\b' + size + r'$', f'EU{size}', text)
    return text
    
result_df['CustomerProductionInstruction'] = df['CustomerProductionInstruction']  #

# 14. SKUNo (from df — added before SpecialRemarks)
result_df['SKUNo'] = df['SKUNo']

# 15. Diamond Quality (from df — needed for remarks)
result_df['Diamond Quality'] = df['Diamond Quality']

# 16. SpecialRemarks
result_df['SpecialRemarks'] = result_df.apply(create_special_remarks, axis=1)

# 17. DesignProductionInstruction
result_df['DesignProductionInstruction'] = result_df['Tone'].apply(get_design_production_instruction)

# 18. StampInstruction
result_df['StampInstruction'] = result_df['Metal'].apply(get_stamp_instruction)

# 19. OrderGroup (blank)
result_df['OrderGroup'] = ""

# 20. Certificate (blank)
result_df['Certificate'] = ""

# 21-28. Additional blank columns
blank_columns = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]
for col in blank_columns:
    result_df[col] = ""


# --- Save and Display ---
output_file = "MOR_Processed_Data2.xlsx"
result_df.to_excel(output_file, index=False)

print("\n✅ Data processed and saved successfully!")
print(f"Output file: {output_file}")
print(f"Total rows: {len(result_df)} | Total columns: {len(result_df.columns)}")
print("\nColumn names:")
for i, col in enumerate(result_df.columns, 1):
    print(f"{i}. {col}")


Please provide the following inputs:


ItemPoNo:  60147686
Priority:  REG



✅ Data processed and saved successfully!
Output file: MOR_Processed_Data2.xlsx
Total rows: 1 | Total columns: 28

Column names:
1. SrNo
2. StyleCode
3. ItemSize
4. OrderQty
5. OrderItemPcs
6. Metal
7. Tone
8. ItemPoNo
9. ItemRefNo
10. StockType
11. Priority
12. MakeType
13. CustomerProductionInstruction
14. SKUNo
15. Diamond Quality
16. SpecialRemarks
17. DesignProductionInstruction
18. StampInstruction
19. OrderGroup
20. Certificate
21. Basestoneminwt
22. Basestonemaxwt
23. Basemetalminwt
24. Basemetalmaxwt
25. Productiondeliverydate
26. Expecteddeliverydate
27. SetPrice
28. StoneQuality
